# Clasificación de Fotos de Casas usando K-Means

## Clasificación automática de imágenes de habitaciones en subdirectorios
 
Este notebook clasifica fotos de partes de una casa (cocina, baño, dormitorio, etc.) en subdirectorios usando:
    - **Extracción de características** con Redes Neuronales Convolucionales (VGG16)
    - **Clustering** con algoritmo K-Means
    - **Organización automática** de archivos

## 1. Instalación de Dependencias

Primero instalamos las librerías necesarias:


In [ ]:
# %pip install numpy pillow scikit-learn tensorflow matplotlib

## 2. Importación de Librerías

In [ ]:
import os
import shutil
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

## 3. Configuración Inicial

Definimos los parámetros principales:


In [ ]:
# Configuración
INPUT_DIR = "./fotos_casa"    # Directorio con fotos originales
OUTPUT_DIR = "./clasificadas" # Directorio de salida
NUM_CLUSTERS = 4              # Número de habitaciones diferentes a clasificar
IMG_SIZE = (224, 224)         # Tamaño requerido por VGG16

# 4. Carga y Preprocesamiento de Imágenes
Función para cargar imágenes y convertirlas al formato requerido por VGG16:

In [ ]:
def cargar_imagenes():
    imagenes = []
    rutas_validas = []
    
    print(f"Cargando imágenes de {INPUT_DIR}...")
    
    for archivo in os.listdir(INPUT_DIR):
        ruta = os.path.join(INPUT_DIR, archivo)
        if not archivo.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
            
        try:
            img = Image.open(ruta).convert('RGB') # Convertir a RGB
            img = img.resize(IMG_SIZE) # Redimensionar a 224x224
            img_array = img_to_array(img) # Convertir a array
            imagenes.append(img_array) # Añadir a la lista
            rutas_validas.append(ruta) # Guardar ruta válida
            print(f"Imagen cargada: {ruta}")
        except Exception as e:
            print(f"Error en {archivo}: {str(e)}")
    
    return np.array(imagenes), rutas_validas

In [ ]:
imagenes, rutas = cargar_imagenes()
print(f"Imágenes cargadas: {len(imagenes)}")

## 5. Extracción de Características
Usamos VGG16 para obtener características profundas de las imágenes:

In [ ]:
def extraer_caracteristicas(imagenes):
    print("Extrayendo características con VGG16...")
    modelo = VGG16(weights='imagenet', include_top=False, pooling='avg') # Modelo VGG16 sin la capa superior
    imagenes_procesadas = preprocess_input(imagenes) # Preprocesar imágenes
    caracteristicas = modelo.predict(imagenes_procesadas) # Extraer características
    return caracteristicas.reshape(len(imagenes), -1) # Aplanar las características

In [ ]:
caracteristicas = extraer_caracteristicas(imagenes)
print(f"Dimensiones de las características: {caracteristicas.shape}")


## 6. Clustering con K-Means
Agrupamos las imágenes en clusters basados en sus características:

In [ ]:
print("Ejecutando K-Means...")
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42)
etiquetas = kmeans.fit_predict(caracteristicas)

# Mostrar distribución de clusters
plt.hist(etiquetas, bins=NUM_CLUSTERS)
plt.title("Distribución de Clusters")
plt.xlabel("Cluster ID")
plt.ylabel("Número de Imágenes")
plt.show()



## 7. Organización de Archivos
Copiamos las imágenes a subdirectorios según su cluster:

In [ ]:
def organizar_imagenes(etiquetas):
    print("Organizando imágenes...")
    
    # Crear directorios
    for i in range(NUM_CLUSTERS):
        os.makedirs(os.path.join(OUTPUT_DIR, f'cluster_{i}'), exist_ok=True)
    
    # Copiar archivos
    for ruta, etiqueta in zip(rutas, etiquetas):
        nombre_archivo = os.path.basename(ruta)
        destino = os.path.join(OUTPUT_DIR, f'cluster_{etiqueta}', nombre_archivo)
        shutil.copy(ruta, destino)
    
    print(f"Organización completada en {OUTPUT_DIR}")

In [ ]:
organizar_imagenes(etiquetas)


## 8. Visualización de Resultados
Mostramos ejemplos de cada cluster:

In [ ]:
def mostrar_ejemplos(n=5):
    fig, axs = plt.subplots(NUM_CLUSTERS, n, figsize=(20, 10))
    
    for cluster in range(NUM_CLUSTERS):
        cluster_dir = os.path.join(OUTPUT_DIR, f'cluster_{cluster}')
        imagenes_cluster = [os.path.join(cluster_dir, f) for f in os.listdir(cluster_dir)][:n]
        
        for i, img_path in enumerate(imagenes_cluster):
            img = Image.open(img_path)
            axs[cluster, i].imshow(img)
            axs[cluster, i].axis('off')
        axs[cluster, 0].set_ylabel(f'Cluster {cluster}', rotation=0, ha='right')
    
    plt.tight_layout()
    plt.show()


In [ ]:
mostrar_ejemplos()

In [ ]:
# %% [markdown]
# ## Pasos Posteriores
# 1. **Revisar manualmente** los clusters generados
# 2. **Renombrar** los directorios `cluster_0`, `cluster_1`, etc. con los nombres reales (ej. "cocina", "baño")
# 3. Ajustar `NUM_CLUSTERS` si se necesitan más/menos categorías
# 4. Probar con diferentes modelos de extracción de características (ResNet, Inception)